In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import PIL
import json
from urbanComponent import *
import math
from vedo import download, Points, Axes, show

from persim import plot_diagrams
from ripser import ripser, lower_star_img

c:\Users\Gustavo\anaconda3\envs\urbantk\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
f = open('../../../public/data/manhattan_wo_cells/buildings_s_joined.json', 'r')

data = json.load(f)

joinedLayerIndex = 0

values = []

for joinedObject in data['joinedObjects']:
    if(joinedObject['joinedLayerIndex'] == joinedLayerIndex):
        values = joinedObject['otherValues']
        break

f.close()

uc = UrbanComponent()

uc.setWorkDir("../../../public/data/manhattan_wo_cells")
uc.addLayerFromJsonFile("../../../public/data/manhattan_wo_cells/buildings_s.json")

buildingsJson = uc.layers['json'][0]

# f = open('./buildings.json', 'w')

# f.write(json.dumps(buildingsJson))

# f.close()

latitude = []
longitude = []
height = []

for geometry in buildingsJson['data']:
    for i in range(int(len(geometry['geometry']['coordinates'])/3)):
        latitude.append(geometry['geometry']['coordinates'][i*3])
        longitude.append(geometry['geometry']['coordinates'][i*3+1])
        height.append(geometry['geometry']['coordinates'][i*3+2])

df = pd.DataFrame(data={'latitude': latitude, 'longitude': longitude, 'height': height})

m1 = df[['latitude','longitude','height']].to_numpy()[:,1]
m2 = df[['latitude','longitude','height']].to_numpy()[:,0]
m3 = df[['latitude','longitude','height']].to_numpy()[:,2]

xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()
zmin = m3.min()
zmax = m3.max()

values = np.array(values).reshape((len(values),))

wmin = values.min()
wmax = values.max()

# X, Y, Z, W = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:50, wmin:wmax:0.1]

X, Y, Z = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:50]

# positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()]) # grid positions

positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]) # grid positions

# data_points = np.vstack([m1, m2, m3, values]) 

data_points = np.vstack([m1, m2, m3]) 

kernel = scipy.stats.gaussian_kde(data_points, weights=values)

# sampled_values = np.reshape(kernel(positions).T, X.shape)
sampled_values = kernel(positions).T


In [3]:
flat_grid_positions = []

for i in range(len(positions[0,:])):
    flat_grid_positions.append(positions[1,i])
    flat_grid_positions.append(positions[0,i])
    flat_grid_positions.append(positions[2,i])

grid_points = {'grid': flat_grid_positions, 'values': sampled_values.tolist()}

json_object = json.dumps(grid_points)

file = open('grid_points.json', 'w')

file.write(json_object)

file.close()

file = open('grid_values.json', 'w')

thematic_layer = {
    "id": "grid_values",
    "coordinates": flat_grid_positions,
    "values": sampled_values.tolist()
}

file.write(json.dumps(thematic_layer))

file.close()


In [13]:
positions[0,:]

array([4940393.3058, 4940393.3058, 4940393.3058, ..., 4941893.3058,
       4941893.3058, 4941893.3058])

### Computing persistence diagram

In [5]:
# dgm = lower_star_img(-reshape_sampled_values)
# dgm[~np.isinf(dgm)] = -dgm[~np.isinf(dgm)]
# dgm[~np.isinf(dgm).any(axis=1)] = np.flip(dgm[~np.isinf(dgm).any(axis=1)])
# # plt.figure(figsize=(10, 5))
# # plt.subplot(121)
# # plt.imshow(np.rot90(Z), cmap=plt.cm.YlOrBr, extent=[xmin, xmax, ymin, ymax])
# # plt.colorbar()
# # plt.title("Scalar function")
# # plt.subplot(122)
# plot_diagrams(dgm)
# plt.title("Persistence diagram")
# plt.tight_layout()
# plt.show()

In [6]:
# threshold = 1.5e-8
# idxs = np.arange(dgm.shape[0])
# idxs = idxs[np.abs(dgm[:, 1] - dgm[:, 0]) > threshold]

# X, Y = np.meshgrid(np.arange(reshape_sampled_values.shape[1]), np.arange(reshape_sampled_values.shape[0]))
# X = X.flatten()
# Y = Y.flatten()
# for idx in idxs:
#     bidx = np.argmin(np.abs(reshape_sampled_values + dgm[idx, 0]))
#     plt.scatter(X[bidx], Y[bidx], 20, 'k')

### 2D plot

In [7]:
# fig, ax = plt.subplots()
# # ax.imshow(np.rot90(reshape_sampled_values), cmap=plt.cm.YlOrBr, extent=[xmin, xmax, ymin, ymax])
# ax.imshow(np.rot90(sampled_values), cmap=plt.cm.YlOrBr)
# #  ax.set_xlim([xmin, xmax])
# # ax.set_ylim([ymin, ymax])
# plt.show()

### 3D plot

In [8]:
# from vedo import download, Points, Axes, show
# from vedo.pyplot import CornerHistogram

# # pts = np.c_[sampled_values[0,:,:],sampled_values[:,0,:],sampled_values[:,:,0]] # form an array of 3d points from the columns

# # pts = Points(positions[0,:].tolist(), positions[1,:].tolist(), positions[2,:].tolist())
# pts = Points(positions)

# pts.pointdata["scals"] = sampled_values.flatten()

# vol = pts.tovolume(kernel='shepard', N=4, dims=(90,90,90))

# vol.c(["maroon","g","b"])        # set color   transfer function
# vol.alpha([0.3, 0.9])            # set opacity transfer function
# #vol.alpha([(0.3,0.3), (0.9,0.9)]) # alternative way, by specifying (xscalar, alpha)
# # vol.alpha_unit(0.5)              # make the whole object less transparent (default is 1)

# # replace voxels of specific range with a new value
# # vol.threshold(above=0.3, below=0.4, replace=0.9)
# # Note that scalar range now has changed (you may want to reapply vol.c().alpha())

# ch = CornerHistogram(vol, pos="bottom-left")

# # vol.add_scalarbar3d('Height is the voxel scalar', s=[None,1])
# # vol.scalarbar.rotate_x(90).pos(1.15,1,0.5)

# show(pts, vol, ch, __doc__, axes=1, elevation=-90).close()

### Creates points layer from best points

In [9]:
# def break_into_binary(filepath, filename, data, types, dataTypes):

#     for index, type in enumerate(types):

#         readCoords = 0

#         floatList = []

#         for i in range(len(data['data'])):
#             geometry = data['data'][i]['geometry']

#             newValue = [readCoords, len(geometry[type])] # where this vector starts and its size

#             readCoords += len(geometry[type])

#             floatList += geometry[type].copy()

#             geometry[type] = newValue

#         fout = open(os.path.join(filepath,filename+'_'+type+'.data'), 'wb')

#         buf = struct.pack(str(len(floatList))+dataTypes[index], *floatList)

#         fout.write(buf)
#         fout.close()

#         json_object = json.dumps(data)

#         with open(os.path.join(filepath,filename+".json"), "w") as outfile:
#             outfile.write(json_object)

In [10]:
# file = open('grid_points.json', 'r')

# grid_points = json.load(file)

# file.close()

# id = 'grid_points'
# type = 'POINTS_LAYER'
# styleKey = 'building'
# renderStyle = ['FLAT_COLOR']
# selectable = False
# visible = True
# skip = False

# layer = {
#     'id': id,
#     'type': type,
#     'styleKey': styleKey,
#     'renderStyle': renderStyle,
#     'selectable': selectable,
#     'visible': visible,
#     'skip': skip
# }

# layer['data'] = [
#     {
#         'geometry': {
#             'coordinates': grid_points['grid']
#         }
#     }
# ]

# types = []
# dataTypes = []

# if('coordinates' in layer['data'][0]['geometry']):
#     types.append("coordinates")
#     dataTypes.append("d")

# if('normals' in layer['data'][0]['geometry']):
#     types.append("normals")
#     dataTypes.append("f")

# if('indices' in layer['data'][0]['geometry']):
#     types.append("indices")
#     dataTypes.append("I")

# if('ids' in layer['data'][0]['geometry']):
#     types.append("ids")
#     dataTypes.append("I")

# break_into_binary('./', layer['id'], layer, types, dataTypes)
